In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
browser = webdriver.Chrome(executable_path='./chromedriver.exe')
#browser.maximize_window()

**Hàm kiểm tra "text" có phải là số không**

In [3]:
def is_number(text):
    for i in text:
        if i.isalpha():
            return False
    return True

**Hàm kiểm tra element có tồn tại không**

In [4]:
def check_element_exist(class_name):
    try:
        #aqi = driver.find_element_by_class_name('aqi-value__value')
        aqi = browser.find_element_by_class_name(class_name)
    except NoSuchElementException:
        return False
    return True

In [5]:
city_names = []
country_links = []
status_text = []
PM25 = []
PM10 = []
O3 = []
NO2 = []
SO2 = []
CO = []

# Lấy data mục "Air quality and pollution city ranking"

In [3]:
url = browser.get('https://www.iqair.com/world-air-quality-ranking')
html_text = browser.page_source

m = browser.find_elements_by_class_name('over-x-auto')

m2 = browser.find_elements(By.XPATH, '//button')
for i in m2:
    if i.text == 'More':
        i.click()
        break


In [4]:
html_text = browser.page_source
tree = BeautifulSoup(html_text, 'html.parser')

links = tree.find_all('td',{'class' : 'text-left'})

urls = []
for link in links:
    urls.append('https://www.iqair.com'+link.find('a',{'class' : 'link-primary'})['href'])
#urls #list url 96 thành phố

In [9]:
for url in urls:
    browser.get(url)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    title = tree.find('h1',{'class' : 'pagetitle__title ng-star-inserted'}).text
    city = title[15:len(title)]
    print("City: ", city)
    city_names.append(city)
    
    index = tree.find_all('div', {'class': 'pollutant-history-type-buttons ng-star-inserted'})
    
    for j in index:
        
        x = browser.find_elements_by_class_name('mat-button-wrapper')
        for X in x:
            if 'DAILY' in X.text:
                X.click()
                break
    #lay data
        if not check_element_exist('aqi-status__text'):
            status_text.append('NULL')
            PM25.append('NULL')
            PM10.append('NULL')
            O3.append('NULL')
            NO2.append('NULL')
            SO2.append('NULL')
            CO.append('NULL')
        else:
            status = browser.find_element_by_class_name('aqi-status__text')
            status_text.append(status.text)
            print('Level: ', status.text)
            #lay ra danh sach cac button chi so
            button_board = browser.find_element_by_class_name('pollutant-history-type-buttons')
            buttons = button_board.find_elements_by_tag_name('button')
            #bo button AQI
            buttons = buttons[1:]
            for button in buttons:
                print('Button name: ', button.text)
                #click vao tung button
                button.click()
                #lay chi so cuoi cung cua bieu do trong button
                chart = browser.find_element_by_class_name('highcharts-series-group')
                cols = chart.find_elements_by_tag_name('rect')
                element_to_hover = cols[-1]
                ActionChains(browser).move_to_element(element_to_hover).perform()
                values = browser.find_elements_by_tag_name('tspan')
                if len(values) == 1:
                    index_value = 0
                else:
                    index_value = values[-2].text
                    if not is_number(index_value):
                        index_value = 0
                print('Value: ', index_value)
                
                if button.text == 'PM2.5':
                    PM25.append(index_value)
                elif button.text =='PM10':
                    PM10.append(index_value)
                elif button.text == 'O3':
                    O3.append(index_value)
                    print('append')
                elif button.text == 'NO2':
                    NO2.append(index_value)
                elif button.text == 'SO2':
                    SO2.append(index_value)
                elif button.text == 'CO':
                    CO.append(index_value)
                
            len_status_text = len(status_text)
            if len(PM25) < len_status_text:
                PM25.append('NULL')
            if len(PM10) < len_status_text:
                PM10.append('NULL')
            if len(O3) < len_status_text:
                O3.append('NULL')
            if len(NO2) < len_status_text:
                NO2.append('NULL')
            if len(SO2) < len_status_text:
                SO2.append('NULL')
            if len(CO) < len_status_text:
                CO.append('NULL')
    print("\n")
print('Finish')

Delhi
Level:  Hazardous
button name:  PM2.5
value: :  318.5
button name:  PM10
value: :  451.4
button name:  O3
value: :  38.2
append
button name:  NO2
value: :  60.7
button name:  SO2
value: :  22.6
button name:  CO
value: :  2317.9
Bishkek
Level:  Hazardous
button name:  PM2.5
value: :  255.3
button name:  PM10
value: :  2.2
Lahore
Level:  Hazardous
button name:  PM2.5
value: :  223.4
Zagreb
Level:  Unhealthy
button name:  PM2.5
value: :  65.1
button name:  PM10
value: :  41.5
button name:  O3
value: :  12.6
append
button name:  NO2
value: :  47.2
button name:  SO2
value: :  5.2
button name:  CO
value: :  940.9
Hanoi
Level:  Unhealthy
button name:  PM2.5
value: :  128.4
button name:  PM10
value: :  189
button name:  O3
value: :  0.8
append
button name:  NO2
value: :  34.4
button name:  SO2
value: :  0.7
button name:  CO
value: :  1847.2
Kolkata
Level:  Unhealthy
button name:  PM2.5
value: :  106.2
button name:  PM10
value: :  188.7
button name:  O3
value: :  65.6
append
button name: 

Level:  Moderate
button name:  PM2.5
value: :  21.5
button name:  PM10
value: :  36.8
button name:  O3
value: :  70.4
append
button name:  NO2
value: :  58.9
button name:  SO2
value: :  5.1
button name:  CO
value: :  857.7
Hong Kong
Level:  Moderate
button name:  PM2.5
value: :  21.9
button name:  PM10
value: :  43.4
button name:  O3
value: :  57.8
append
button name:  NO2
value: :  93.9
button name:  SO2
value: :  6
button name:  CO
value: :  565.1
Kyoto
Level:  Moderate
button name:  PM2.5
value: :  20.8
button name:  PM10
value: :  38.6
button name:  NO2
value: :  61.4
button name:  CO
value: :  636.1
Mexico City
Level:  Moderate
button name:  PM2.5
value: :  15.3
button name:  PM10
value: :  33
button name:  O3
value: :  16.3
append
button name:  NO2
value: :  38.2
button name:  SO2
value: :  13.5
button name:  CO
value: :  847.3
Nur-Sultan
Level:  Moderate
button name:  PM2.5
value: :  20.4
button name:  PM10
value: :  68.8
button name:  NO2
value: :  25.4
button name:  SO2
value:

value: :  49.2
append
button name:  NO2
value: :  37.2
button name:  SO2
value: :  0
button name:  CO
value: :  204.5
Skopje
Level:  Good
button name:  PM2.5
value: :  10.2
button name:  PM10
value: :  12.5
button name:  O3
value: :  32
append
button name:  NO2
value: :  14.1
button name:  SO2
value: :  2.3
Melbourne
Level:  Good
button name:  PM2.5
value: :  2
button name:  PM10
value: :  22.1
button name:  O3
value: :  13.5
append
button name:  NO2
value: :  3.3
button name:  SO2
value: :  4.2
button name:  CO
value: :  0
Salt Lake City
Level:  Good
button name:  PM2.5
value: :  1.6
button name:  O3
value: :  36
append
button name:  NO2
value: :  64.5
button name:  SO2
value: :  0
button name:  CO
value: :  57.3


# Lấy data mục "Air quality in the world"

In [6]:
browser.get('https://www.iqair.com/us/world-air-quality')
board = browser.find_element_by_class_name('aqi-ranking')
countries = board.find_elements_by_tag_name('a')

#bo button SEE DETAIL o cuoi cung
countries = countries[:-1]
#countries = countries[:1]

for country in countries:
    #lay ra duong link dan den cac quoc gia trong top
    country_link = board.find_element_by_link_text(country.text).get_attribute('href')
    country_links.append(country_link)
    
for country_link in country_links:
    print('country: ', country_link)
    while True:        
        browser.get(country_link)
        #lay ra mot so thanh pho cua country
        if not check_element_exist('ranking__table'):
            break
            
        ranking_table = browser.find_element_by_class_name('ranking__table')
        cities = ranking_table.find_elements_by_tag_name('a')
        
        remain_city = False #kiem tra xem con thanh pho nao chua duoc duyet hay khong
        for city in cities:
            if city.text not in city_names:
                remain_city = True
                
                city_names.append(city.text)
                print('city name: ', city.text)
                city_link = city.get_attribute('href')
                browser.get(city_link)
        
                #lay ra muc do chat luong khong khi
                if not check_element_exist('aqi-status__text'):
                    status_text.append('NULL')
                    PM25.append('NULL')
                    PM10.append('NULL')
                    O3.append('NULL')
                    NO2.append('NULL')
                    SO2.append('NULL')
                    CO.append('NULL')
                else:
                    status = browser.find_element_by_class_name('aqi-status__text')
                    status_text.append(status.text)
                    print('Level: ', status.text)
                    #lay ra danh sach cac button chi so
                    button_board = browser.find_element_by_class_name('pollutant-history-type-buttons')
                    buttons = button_board.find_elements_by_tag_name('button')
                    #bo button AQI
                    buttons = buttons[1:]
                    for button in buttons:
                        print('button name: ', button.text)
                        #click vao tung button
                        button.click()
                        #lay chi so cuoi cung cua bieu do trong button
                        chart = browser.find_element_by_class_name('highcharts-series-group')
                        cols = chart.find_elements_by_tag_name('rect')
                        element_to_hover = cols[-1]
                        ActionChains(browser).move_to_element(element_to_hover).perform()
                        values = browser.find_elements_by_tag_name('tspan')
                        if len(values) == 1:
                            index_value = 0
                        else:
                            index_value = values[-2].text
                            if not is_number(index_value):
                                index_value = 0
                                
                        print(index_value)
                        
                        if button.text == 'PM2.5':
                            PM25.append(index_value)
                        elif button.text =='PM10':
                            PM10.append(index_value)
                        elif button.text == 'O3':
                            O3.append(index_value)
                            print('append')
                        elif button.text == 'NO2':
                            NO2.append(index_value)
                        elif button.text == 'SO2':
                            SO2.append(index_value)
                        elif button.text == 'CO':
                            CO.append(index_value)
                        
                    len_status_text = len(status_text)
                    if len(PM25) < len_status_text:
                        PM25.append('NULL')
                    if len(PM10) < len_status_text:
                        PM10.append('NULL')
                    if len(O3) < len_status_text:
                        O3.append('NULL')
                    if len(NO2) < len_status_text:
                        NO2.append('NULL')
                    if len(SO2) < len_status_text:
                        SO2.append('NULL')
                    if len(CO) < len_status_text:
                        CO.append('NULL')
                break
        #neu tat ca cac thanh pho da duoc duyet thi ket thuc
        if remain_city == False:
            break
print('finish')

country:  https://www.iqair.com/us/bangladesh
city name:  Sreepur, Dhaka
Level:  Very Unhealthy
button name:  PM2.5
223.7
city name:  Dhaka, Dhaka
Level:  Very Unhealthy
button name:  PM2.5
199.4
city name:  Manikganj, Dhaka
Level:  Very Unhealthy
button name:  PM2.5
175.2
city name:  Savar, Dhaka
Level:  Unhealthy
button name:  PM2.5
113.1
city name:  Narayanganj, Dhaka
Level:  Good
button name:  PM2.5
10
country:  https://www.iqair.com/us/pakistan
city name:  Faisalabad, Punjab
Level:  Hazardous
button name:  PM2.5
313.4
city name:  Lahore, Punjab
Level:  Hazardous
button name:  PM2.5
257.5
city name:  Rawalpindi, Punjab
Level:  Very Unhealthy
button name:  PM2.5
177.1
city name:  Raiwind, Punjab
Level:  Very Unhealthy
button name:  PM2.5
159.1
city name:  Islamabad, Islamabad
Level:  Unhealthy
button name:  PM2.5
100.4
city name:  Muridke, Punjab
Level:  Unhealthy
button name:  PM2.5
83.4
city name:  Multan, Punjab
Level:  Unhealthy
button name:  PM2.5
66.3
city name:  Abbottabad, K

ElementClickInterceptedException: Message: element click intercepted: Element <button _ngcontent-airvisual-web-c186="" type="button" mat-flat-button="" color="primary" class="mat-focus-indicator btn-airvisual-2 mat-flat-button mat-button-base mat-primary ng-star-inserted">...</button> is not clickable at point (432, 532). Other element would receive the click: <iframe src="https://oc-cdn-ocprod.azureedge.net/livechatwidget/WebChatControl/htmls/chat.html?&amp;id=Microsoft_Omnichannel_LCWidget&amp;data-app-id=5ac1541d-b8e5-47cd-af83-85680b73f649&amp;data-org-id=a4c11228-cbc9-4731-9eb5-deca162c991f&amp;data-org-url=https://orgc5ee6eb2-crm.omnichannelengagementhub.com&amp;hostname=www.iqair.com&amp;data-hide-chat-button=false&amp;data-lcw-version=prod" id="Microsoft_Omnichannel_LCWidget_Chat_Iframe_Window" class="start_chat_button BottomRight" title="Microsoft Omnichannel Live Chat Widget" frameborder="0px" allow="microphone *; camera *; geolocation *;" allowfullscreen="true" mozallowfulls

# Lấy data một số quốc gia khác

In [13]:
list_countries_str = 'japan china canada vietnam uk france germany thailand switzerland findland greece brazil russia philippines myanmar laos usa bulgaria malaysia peru serbia slovakia spain syria'
list_countries = list_countries_str.split()

for country in list_countries:
    #lay ra duong link dan den cac quoc gia trong top
    country_link = 'https://www.iqair.com/' + country
    country_links.append(country_link)
    
for country_link in country_links:
    print('country: ', country_link)
    while True:        
        browser.get(country_link)
        #lay ra mot so thanh pho cua country
        if not check_element_exist('ranking__table'):
            break
            
        ranking_table = browser.find_element_by_class_name('ranking__table')
        cities = ranking_table.find_elements_by_tag_name('a')
        
        remain_city = False #kiem tra xem con thanh pho nao chua duoc duyet hay khong
        for city in cities:
            if city.text not in city_names:
                remain_city = True
                
                city_names.append(city.text)
                print('city name: ', city.text)
                city_link = city.get_attribute('href')
                browser.get(city_link)
        
                #lay ra chi so AQI
                if not check_element_exist('aqi-status__text'):
                    status_text.append('NULL')
                    PM25.append('NULL')
                    PM10.append('NULL')
                    O3.append('NULL')
                    NO2.append('NULL')
                    SO2.append('NULL')
                    CO.append('NULL')
                else:
                    status = browser.find_element_by_class_name('aqi-status__text')
                    status_text.append(status.text)
                    #lay ra danh sach cac button chi so
                    button_board = browser.find_element_by_class_name('pollutant-history-type-buttons')
                    buttons = button_board.find_elements_by_tag_name('button')
                    #bo button AQI
                    buttons = buttons[1:]
                    for button in buttons:
                        print('button name: ', button.text)
                        #click vao tung button
                        button.click()
                        #lay chi so cuoi cung cua bieu do trong button
                        chart = browser.find_element_by_class_name('highcharts-series-group')
                        cols = chart.find_elements_by_tag_name('rect')
                        element_to_hover = cols[-1]
                        ActionChains(browser).move_to_element(element_to_hover).perform()
                        values = browser.find_elements_by_tag_name('tspan')
                        if len(values) == 1:
                            index_value = 0
                        else:
                            index_value = values[-2].text
                            if not is_number(index_value):
                                index_value = 0
                                
                        print(index_value)
                        
                        if button.text == 'PM2.5':
                            PM25.append(index_value)
                        elif button.text =='PM10':
                            PM10.append(index_value)
                        elif button.text == 'O3':
                            O3.append(index_value)
                        elif button.text == 'NO2':
                            NO2.append(index_value)
                        elif button.text == 'SO2':
                            SO2.append(index_value)
                        elif button.text == 'CO':
                            CO.append(index_value)
                        
                    len_status_text = len(status_text)
                    if len(PM25) < len_status_text:
                        PM25.append('NULL')
                    if len(PM10) < len_status_text:
                        PM10.append('NULL')
                    if len(O3) < len_status_text:
                        O3.append('NULL')
                    if len(NO2) < len_status_text:
                        NO2.append('NULL')
                    if len(SO2) < len_status_text:
                        SO2.append('NULL')
                    if len(CO) < len_status_text:
                        CO.append('NULL')
                    
                break
        #neu tat ca cac thanh pho da duoc duyet thi ket thuc
        if remain_city == False:
            break
print('finish')

country:  https://www.iqair.com/us/bangladesh
country:  https://www.iqair.com/us/pakistan
country:  https://www.iqair.com/us/mongolia
city name:  Urt, OEmnoegovi
button name:  PM2.5
21
button name:  PM10
49
button name:  NO2
7
button name:  SO2
6
city name:  Ulaan-Uul, East Gobi Aymag
button name:  PM2.5
22
button name:  PM10
28
button name:  NO2
34
button name:  SO2
70
city name:  Tsetserleg, Arhangay
button name:  PM2.5
12.7
city name:  Sumber, East Aimak
button name:  PM2.5
9
button name:  PM10
10
button name:  NO2
10
button name:  SO2
6
city name:  Chonogol, Suhbaatar
button name:  PM2.5
6
button name:  PM10
36
button name:  NO2
23
button name:  SO2
19
city name:  Hongor, Suhbaatar
button name:  PM2.5
10
button name:  PM10
16
button name:  NO2
5
button name:  SO2
8
city name:  Ulan Bator, Ulaanbaatar
country:  https://www.iqair.com/us/afghanistan
city name:  Kabul, Kabul
button name:  PM2.5
14.9
country:  https://www.iqair.com/us/india
city name:  Alipur, Delhi
button name:  PM2.5


ElementClickInterceptedException: Message: element click intercepted: Element <button _ngcontent-airvisual-web-c186="" type="button" mat-flat-button="" color="primary" class="mat-focus-indicator btn-airvisual-2 mat-flat-button mat-button-base mat-primary ng-star-inserted">...</button> is not clickable at point (1060, 542). Other element would receive the click: <iframe src="https://oc-cdn-ocprod.azureedge.net/livechatwidget/WebChatControl/htmls/chat.html?&amp;id=Microsoft_Omnichannel_LCWidget&amp;data-app-id=5ac1541d-b8e5-47cd-af83-85680b73f649&amp;data-org-id=a4c11228-cbc9-4731-9eb5-deca162c991f&amp;data-org-url=https://orgc5ee6eb2-crm.omnichannelengagementhub.com&amp;hostname=www.iqair.com&amp;data-hide-chat-button=false&amp;data-lcw-version=prod" id="Microsoft_Omnichannel_LCWidget_Chat_Iframe_Window" class="start_chat_button BottomRight" title="Microsoft Omnichannel Live Chat Widget" frameborder="0px" allow="microphone *; camera *; geolocation *;" allowfullscreen="true" mozallowfull

In [14]:
data = {'City': city_names, 'Level': status_text, 'PM2.5': PM25, 'PM10': PM10, 'O3': O3, 'NO2': NO2, 'SO2':SO2, 'CO': CO}
df = pd.DataFrame(data)


ValueError: arrays must all be same length

**Xem thử dataframe**

In [19]:
df.head()

,City,Level,PM2.5,PM10,O3,NO2,SO2,CO
0,Dhaka,Very Unhealthy,227.5,NULL,NULL,NULL,NULL,NULL
1,Delhi,Very Unhealthy,137.7,201.2,9.8,35,12.6,1181.7
2,Bishkek,Very Unhealthy,157.4,2.2,NULL,NULL,NULL,NULL
3,Kathmandu,Unhealthy,112.2,198.5,14.8,NULL,NULL,NULL
4,Lahore,Unhealthy,104.5,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...
91,Tokyo,Good,6.1,6.4,NULL,44.2,2.6,381.7
92,Los Angeles,Good,17.1,41.2,3,31.8,2.4,NULL
93,Sydney,Good,0,7.2,31.5,1.1,0,2.3
94,Skopje,Good,6.8,4.9,20.3,11.5,0,NULL


In [20]:
df.to_csv("data.csv")